In [1]:
from datetime import datetime
from ib_insync import * 
import random
import math
import pytz
import ntplib
import time
import tests
import ccxt
import sqlalchemy
import pandas as pd
import numpy as np
from os.path import exists
from pykalman import KalmanFilter

#development:4aX!m{rR[4OQY4q

#from dydx3 import Client
from web3 import Web3
import pandas as pd

ticker1= 'BTCUSDT'
bar_size_lookback = '5m'
exchange = ccxt.binanceusdm()

In [2]:
def get_prices_crypto(ticker, bar_size, bars, return_all_data=False):
    while True:  
        try: 
            if return_all_data == True: 
                data = exchange.fetch_ohlcv(ticker,bar_size,limit=bars)
                return data
            else:
                data = exchange.fetch_ohlcv(ticker,bar_size,limit=bars)
                prices = []

                for d in data: 
                    prices.append(d[4])
                return prices
        except Exception as e: 
            print(e)
            time.sleep(10)

In [3]:
print('1111')
latest_time_pulled = get_prices_crypto(ticker1, bar_size_lookback, 2, return_all_data=True) #*** need to change the time of the last bar from binace
delta_time = int(abs((latest_time_pulled[0][0] - latest_time_pulled[1][0])/1000)) #in seconds

print(latest_time_pulled)
print(delta_time)


1111
[[1664083800000, 18944.6, 18964.9, 18942.0, 18952.6, 815.797], [1664084100000, 18952.6, 18984.7, 18942.1, 18963.0, 1174.989]]
300


In [ ]:
print(1)

In [ ]:
def update_global_data_crypto(ticker1, ticker2): 
    global global_data, global_time, hedge_ratio, duration_lookback, bar_size_lookback
    duration = duration_lookback
    update = False

#     if len(global_data) == 0:  
#         print('Global data is empty, initializing data!')
#         replace_df = True
#     else:
        latest_time_pulled = get_prices_crypto(ticker1, bar_size_lookback, 2, return_all_data=True) #*** need to change the time of the last bar from binace
        delta_time = int(abs((latest_time_pulled[0][0] - latest_time_pulled[1][0])/1000)) #in seconds


In [ ]:

        #get the latest time in the df
        latest_df_time = global_data['Time'].iloc[0]
        differential = latest_time_pulled[-1][0]- latest_df_time
        if differential > 0: 
            delta_bars = ((latest_time_pulled[-1][0] - latest_df_time)/(delta_time*1000))*1.2
            if delta_bars > len(global_data): 
                replace_df = True #update entire dataframe
            else: 
                replace_df = False #updates necessary part of dataframe
                duration = delta_bars
        elif differential == 0: 
            update = True
            duration = 1
            replace_df = False

    data1 = get_prices_crypto(ticker1,bar_size_lookback, math.ceil(duration),return_all_data=True) #***data needs to be looked at and adjusted 
    data2 = get_prices_crypto(ticker2,bar_size_lookback, math.ceil(duration),return_all_data=True)

    times = [data1[i][0] for i in range(len(data1))]

    data1, data2, times = data1[::-1], data2[::-1], times[::-1] #needed to reverse list

    data1_close = [data1[i][4] for i in range(len(data1))]
    data2_close = [data2[i][4] for i in range(len(data2))]

    data1_low = [data1[i][3] for i in range(len(data1))]
    data2_low = [data2[i][3] for i in range(len(data2))]

    data1_high = [data1[i][2] for i in range(len(data1))]
    data2_high = [data2[i][2] for i in range(len(data2))]

    spread = []
    for i in range(len(data1_close)):
        if spread_type == 'log':
            spread.append(math.log(data1_close[i]) - hedge_ratio*math.log(data2_close[i]))
        elif spread_type == 'absolute':
            spread.append(data1_close[i] - hedge_ratio*data2_close[i])

    df = pd.DataFrame({'Time':times, 'Ticker 1 Price':data1_close,'Ticker 1 Low':data1_low, 'Ticker 1 High':data1_high,'Ticker 2 Price':data2_close, 'Ticker 2 Low':data2_low, 'Ticker 2 High':data2_high,'Spread': spread})
 
    if update == True: 
        global_data.iloc[0] = df.iloc[0]

    if replace_df == False and update == False: 
        df_time = df['Time'].iloc[-1]
        rows = 0
        for t in global_data['Time']:
            rows += 1
            if df_time == t: 
                break 
            
        if rows > 0: 
            global_data = global_data.truncate(before=rows)
            #concatenate 
            global_data = pd.merge(df,global_data, how="outer")

    elif replace_df == True:
        global_data = df

    global_data = global_data[:global_data_max_size]